In [1]:
import tweepy
import csv
import time
import html.parser as hp
import pandas as pd
import nltk
import pickle
import numpy as np
import gensim
import re  # regular expressions (for playing with the text)
from gensim.parsing.preprocessing import STOPWORDS # common english "stop words" -- a, the, etc.
from gensim.parsing import PorterStemmer
from gensim import corpora, models
import nltk.data
import nltk

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import spacy

from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import gensim
import os
from os import path
from time import sleep
import matplotlib.pyplot as plt
import random
import wordcloud
from wordcloud import WordCloud , STOPWORDS

import re
import pickle
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")



In [2]:
X_train=pd.read_csv('TrainData.csv')

In [3]:
X_train.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
X_train.shape

(5394, 2)

In [5]:
X_val=pd.read_csv('ValidationData.csv')

In [6]:
X_val.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
X_val.shape

(1349, 2)

In [8]:
y_train = X_train.Sentiment.values

In [9]:
y_train.shape

(5394,)

In [10]:
y_val=X_val.Sentiment.values

In [11]:
y_val.shape

(1349,)

In [12]:
test=pd.read_csv('Final_Relabelled_8thApril.csv')

In [13]:
test.shape

(1679, 2)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(X_train['Sentences'])
val_vectors = vectorizer.transform(X_val['Sentences'])
test_vectors = vectorizer.transform(test['Sentences'])

## Logistic Regression

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [50]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(train_vectors, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, lr.predict(val_vectors))))

Accuracy for C=0.01: 0.6345441067457376
Accuracy for C=0.05: 0.6419570051890289
Accuracy for C=0.25: 0.6723498888065234
Accuracy for C=0.5: 0.6879169755374351
Accuracy for C=1: 0.6975537435137139


In [51]:
final_tfidf = LogisticRegression(C=1)
final_tfidf.fit(train_vectors, y_train)
print ("Final Accuracy: %s" % accuracy_score(test['Sentiment'], final_tfidf.predict(test_vectors)))

Final Accuracy: 0.7111375818939845


# Linear SVM

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [96]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(train_vectors, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, svm.predict(val_vectors))))
    

Accuracy for C=0.01: 0.6553002223869533
Accuracy for C=0.05: 0.6834692364714603
Accuracy for C=0.25: 0.7057079318013343
Accuracy for C=0.5: 0.7034840622683469
Accuracy for C=1: 0.7049666419570052


In [97]:
final_svm_ngram = LinearSVC(C=0.25)
final_svm_ngram.fit(train_vectors, y_train)
print ("Final Accuracy: %s" % accuracy_score(test['Sentiment'], final_svm_ngram.predict(test_vectors)))

Final Accuracy: 0.6974389517569982


## Light GBM

In [56]:
import lightgbm as lgb
from sklearn import metrics

lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [10,15,20,25],
              "learning_rate" : [0.01,0.05],
              "num_leaves": [100,200,300],
              "n_estimators": [100,200,300]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)

In [ ]:
grid_search.fit(train_vectors, y_train)

In [ ]:
grid_search.best_estimator_

In [94]:
d_train = lgb.Dataset(train_vectors, train_y)

In [95]:
lg = lgb.LGBMClassifier(silent=False)

In [97]:
params = {"boosting_type" :'gbdt', "class_weight" : None, "colsample_bytree" : 1.0,
        "importance_type" : 'split', "learning_rate" : 0.005, "max_depth" : 20,
        "min_child_samples" : 20, "min_child_weight" : 0.001, "min_split_gain" : 0.0,
        "n_estimators" : 300, "n_jobs" : -1, "num_leaves" : 100, "objective" : None,
        "random_state" : None, "reg_alpha" : 0.0, "reg_lambda" : 0.0, "silent" : False,
        "subsample" : 1.0, "subsample_for_bin" : 200000, "subsample_freq" : 0}

In [61]:
#params = {"max_depth": 25, "learning_rate" : 0.1, "num_leaves": 300,  "n_estimators": 200}

In [98]:
model2 = lgb.train(params, d_train)

/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/site-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


In [123]:
pred=model2.predict(test_vectors)

In [124]:
pred

array([0.33708272, 0.41618535, 0.1777202 , ..., 0.41829765, 0.57026489,
       0.73166651])

In [125]:
for i in range(len(pred)):
    if pred[i]>=.5: 
        pred[i]=1
    else:  
        pred[i]=0

In [126]:
print('Accuracy score is : {:.2f}'.format(accuracy_score(test_y,pred)))

Accuracy score is : 0.69


In [127]:
print(confusion_matrix(test_y,pred))

[[1466  169]
 [ 620  278]]


# Implementing UnSupervised

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()

In [16]:
sia.lexicon.clear()

In [17]:
sia.lexicon

{}

In [18]:
new_words = {'abandon' :-2,
'abandoned' :-2,
'abandonment' :-2,
'abnormally' :-2,
'abolished' :-2,
'abrogate' :-2,
'abrupt' :-2,
'abruptly' :-2,
'abuse' :-2,
'abuses' :-2,
'abusive' :-2,
'accident' :-2,
'accidentally' :-2,
'accidents' :-2,
'adversarial' :-2,
'adverse' :-2,
'adversely' :-2,
'adversity' :-2,
'aftermath' :-2,
'alienation' :-2,
'allegations' :-2,
'allege' :-2,
'alleged' :-2,
'anomalies' :-2,
'antitrust' :-2,
'arrearage' :-2,
'arrearages' :-2,
'arrears' :-2,
'artificially' :-2,
'bad' :-2,
'bail' :-2,
'bailout' :-2,
'bankrupt' :-2,
'bankruptcies' :-2,
'bankruptcy' :-2,
'bans' :-2,
'barrier' :-2,
'barriers' :-2,
'bottlenecks' :-2,
'boycotts' :-2,
'breach' :-2,
'breached' :-2,
'breaches' :-2,
'break' :-2,
'breakdown' :-2,
'breakdowns' :-2,
'breaking' :-2,
'breaks' :-2,
'broken' :-2,
'burden' :-2,
'burdened' :-2,
'burdens' :-2,
'burdensome' :-2,
'cancel' :-2,
'canceled' :-2,
'cancellation' :-2,
'cancelled' :-2,
'cancelling' :-2,
'cancels' :-2,
'catastrophe' :-2,
'catastrophes' :-2,
'catastrophic' :-2,
'cease' :-2,
'ceased' :-2,
'ceases' :-2,
'challenge' :-2,
'challenged' :-2,
'challenges' :-2,
'challenging' :-2,
'circumvented' :-2,
'claiming' :-2,
'claims' :-2,
'closed' :-2,
'closing' :-2,
'closings' :-2,
'closure' :-2,
'closures' :-2,
'coerce' :-2,
'collapse' :-2,
'collapsed' :-2,
'colluded' :-2,
'collusion' :-2,
'complaints' :-2,
'complicated' :-2,
'complications' :-2,
'concern' :-2,
'concerned' :-2,
'concerns' :-2,
'confiscation' :-2,
'confiscations' :-2,
'conflict' :-2,
'conflicting' :-2,
'conflicts' :-2,
'confront' :-2,
'confrontations' :-2,
'confronted' :-2,
'confronting' :-2,
'confused' :-2,
'confusion' :-2,
'contend' :-2,
'contention' :-2,
'contentious' :-2,
'contested' :-2,
'contesting' :-2,
'contraction' :-2,
'contradict' :-2,
'contradicted' :-2,
'contradictory' :-2,
'contrary' :-2,
'controversial' :-2,
'controversies' :-2,
'controversy' :-2,
'corrected' :-2,
'correcting' :-2,
'correction' :-2,
'corrections' :-2,
'corrects' :-2,
'corrupting' :-2,
'corruption' :-2,
'costly' :-2,
'counterclaims' :-2,
'counterfeit' :-2,
'countermeasures' :-2,
'crime' :-2,
'crises' :-2,
'crisis' :-2,
'critical' :-2,
'crucial' :-2,
'curtail' :-2,
'curtailed' :-2,
'curtailing' :-2,
'curtailment' :-2,
'curtails' :-2,
'cut' :-2,
'cutbacks' :-2,
'cyberattacks' :-2,
'cybercrime' :-2,
'damage' :-2,
'damaged' :-2,
'damages' :-2,
'damaging' :-2,
'dampen' :-2,
'dampened' :-2,
'danger' :-2,
'dangers' :-2,
'deceptive' :-2,
'decline' :-2,
'declined' :-2,
'declines' :-2,
'declining' :-2,
'default' :-2,
'defaulted' :-2,
'defaulting' :-2,
'defaults' :-2,
'defeat' :-2,
'defect' :-2,
'defective' :-2,
'defects' :-2,
'defend' :-2,
'defending' :-2,
'defensive' :-2,
'defer' :-2,
'deficiencies' :-2,
'deficiency' :-2,
'deficit' :-2,
'deficits' :-2,
'defraud' :-2,
'degradation' :-2,
'delay' :-2,
'delayed' :-2,
'delaying' :-2,
'delays' :-2,
'deliberate' :-2,
'delinquencies' :-2,
'delinquency' :-2,
'delinquent' :-2,
'delisted' :-2,
'delisting' :-2,
'denial' :-2,
'denied' :-2,
'deny' :-2,
'dependence' :-2,
'dependencies' :-2,
'dependency' :-2,
'dependent' :-2,
'deplete' :-2,
'depleted' :-2,
'depleting' :-2,
'depletion' :-2,
'depress' :-2,
'depressed' :-2,
'depresses' :-2,
'depressing' :-2,
'deprive' :-2,
'depriving' :-2,
'destabilize' :-2,
'destabilized' :-2,
'destabilizing' :-2,
'destroy' :-2,
'destroyed' :-2,
'destruction' :-2,
'deter' :-2,
'deteriorate' :-2,
'deteriorated' :-2,
'deteriorates' :-2,
'deteriorating' :-2,
'deterioration' :-2,
'deteriorations' :-2,
'detract' :-2,
'detracted' :-2,
'detriment' :-2,
'detrimental' :-2,
'detrimentally' :-2,
'detriments' :-2,
'devalue' :-2,
'devalued' :-2,
'devastation' :-2,
'deviate' :-2,
'deviated' :-2,
'deviates' :-2,
'deviating' :-2,
'deviation' :-2,
'deviations' :-2,
'difficult' :-2,
'difficulties' :-2,
'difficulty' :-2,
'diminish' :-2,
'diminished' :-2,
'diminishing' :-2,
'diminution' :-2,
'disadvantage' :-2,
'disadvantaged' :-2,
'disadvantageous' :-2,
'disadvantages' :-2,
'disagree' :-2,
'disagreement' :-2,
'disagreements' :-2,
'disallow' :-2,
'disallowances' :-2,
'disappear' :-2,
'disappearance' :-2,
'disappointing' :-2,
'disappointments' :-2,
'disapprove' :-2,
'disapproved' :-2,
'disaster' :-2,
'disasters' :-2,
'disciplinary' :-2,
'discontinuance' :-2,
'discontinuation' :-2,
'discontinues' :-2,
'discourage' :-2,
'discourages' :-2,
'discouraging' :-2,
'discrepancies' :-2,
'discrepancy' :-2,
'disfavor' :-2,
'disgorge' :-2,
'disorderly' :-2,
'disparities' :-2,
'disparity' :-2,
'displace' :-2,
'displacement' :-2,
'dispose' :-2,
'disproportional' :-2,
'disproportionate' :-2,
'disproportionately' :-2,
'dispute' :-2,
'disputed' :-2,
'disputes' :-2,
'disqualification' :-2,
'disqualified' :-2,
'disrupt' :-2,
'disrupted' :-2,
'disruption' :-2,
'disruptions' :-2,
'disruptive' :-2,
'dissolution' :-2,
'distorted' :-2,
'distortions' :-2,
'distress' :-2,
'distressed' :-2,
'disturbances' :-2,
'diversion' :-2,
'divert' :-2,
'divest' :-2,
'divesting' :-2,
'divestitures' :-2,
'doubt' :-2,
'doubtful' :-2,
'doubts' :-2,
'downgrade' :-2,
'downgraded' :-2,
'downgrades' :-2,
'downgrading' :-2,
'downsizing' :-2,
'downtime' :-2,
'downturn' :-2,
'downturns' :-2,
'downward' :-2,
'downwards' :-2,
'drag' :-2,
'drastic' :-2,
'drastically' :-2,
'dropped' :-2,
'drought' :-2,
'droughts' :-2,
'dysfunction' :-2,
'dysfunctions' :-2,
'easing' :-2,
'egregiously' :-2,
'encumber' :-2,
'encumbered' :-2,
'erode' :-2,
'eroded' :-2,
'erodes' :-2,
'eroding' :-2,
'erosion' :-2,
'erratic' :-2,
'erratically' :-2,
'erroneous' :-2,
'erroneously' :-2,
'error' :-2,
'errors' :-2,
'escalate' :-2,
'escalated' :-2,
'escalating' :-2,
'evasion' :-2,
'exacerbate' :-2,
'exacerbated' :-2,
'exacerbates' :-2,
'exacerbating' :-2,
'exaggerate' :-2,
'exaggerated' :-2,
'exaggerates' :-2,
'excessive' :-2,
'excessively' :-2,
'exploit' :-2,
'exploitation' :-2,
'exploiting' :-2,
'expose' :-2,
'exposed' :-2,
'exposes' :-2,
'exposing' :-2,
'expropriate' :-2,
'expropriated' :-2,
'expropriates' :-2,
'expropriating' :-2,
'expropriation' :-2,
'expropriations' :-2,
'fail' :-2,
'failed' :-2,
'failing' :-2,
'fails' :-2,
'failure' :-2,
'failures' :-2,
'fall' :-2,
'fallout' :-2,
'false' :-2,
'fault' :-2,
'faulty' :-2,
'fear' :-2,
'fears' :-2,
'fines' :-2,
'flawed' :-2,
'flaws' :-2,
'force' :-2,
'forced' :-2,
'forcing' :-2,
'foreclose' :-2,
'foreclosed' :-2,
'foreclosure' :-2,
'foreclosures' :-2,
'forego' :-2,
'foregoes' :-2,
'foregone' :-2,
'forfeited' :-2,
'forfeiture' :-2,
'fraud' :-2,
'fraudulent' :-2,
'halt' :-2,
'halted' :-2,
'hamper' :-2,
'hampered' :-2,
'hampers' :-2,
'hardship' :-2,
'harm' :-2,
'harmed' :-2,
'harmful' :-2,
'harms' :-2,
'harsh' :-2,
'harsher' :-2,
'hazardous' :-2,
'hazards' :-2,
'hinder' :-2,
'hindered' :-2,
'hostile' :-2,
'hostility' :-2,
'hurt' :-2,
'hurting' :-2,
'ill' :-2,
'illegal' :-2,
'illiquid' :-2,
'illiquidity' :-2,
'immature' :-2,
'impair' :-2,
'impaired' :-2,
'impairing' :-2,
'impairment' :-2,
'impairments' :-2,
'impairs' :-2,
'impede' :-2,
'impediment' :-2,
'impediments' :-2,
'impending' :-2,
'imperfection' :-2,
'imperfections' :-2,
'impermissible' :-2,
'impossible' :-2,
'impracticable' :-2,
'impractical' :-2,
'improper' :-2,
'improperly' :-2,
'inability' :-2,
'inaccessible' :-2,
'inaccuracies' :-2,
'inaccuracy' :-2,
'inaccurate' :-2,
'inaccurately' :-2,
'inaction' :-2,
'inadequacy' :-2,
'inadequate' :-2,
'inadvertent' :-2,
'inadvertently' :-2,
'inadvisable' :-2,
'inappropriate' :-2,
'incapable' :-2,
'incidence' :-2,
'incident' :-2,
'incidents' :-2,
'incompatibility' :-2,
'inconsistencies' :-2,
'inconsistency' :-2,
'inconsistent' :-2,
'incorrect' :-2,
'incorrectly' :-2,
'ineffective' :-2,
'ineffectively' :-2,
'inefficiencies' :-2,
'inefficiency' :-2,
'inefficient' :-2,
'ineligible' :-2,
'inequitable' :-2,
'inexperience' :-2,
'inexperienced' :-2,
'inferior' :-2,
'infringed' :-2,
'infringement' :-2,
'infringing' :-2,
'inhibited' :-2,
'injunction' :-2,
'injunctions' :-2,
'injuries' :-2,
'injury' :-2,
'inordinately' :-2,
'insolvencies' :-2,
'insolvency' :-2,
'insolvent' :-2,
'instability' :-2,
'insufficiency' :-2,
'insufficient' :-2,
'insufficiently' :-2,
'insurrections' :-2,
'interfere' :-2,
'interfered' :-2,
'interference' :-2,
'interferences' :-2,
'interferes' :-2,
'interfering' :-2,
'interrupt' :-2,
'interrupted' :-2,
'interruption' :-2,
'interruptions' :-2,
'invalid' :-2,
'invalidate' :-2,
'invalidated' :-2,
'invalidates' :-2,
'invalidating' :-2,
'invalidation' :-2,
'involuntarily' :-2,
'involuntary' :-2,
'irregular' :-2,
'irregularities' :-2,
'irregularly' :-2,
'jeopardize' :-2,
'jeopardized' :-2,
'lack' :-2,
'lacked' :-2,
'lacking' :-2,
'lackluster' :-2,
'lacks' :-2,
'lag' :-2,
'lagged' :-2,
'lagging' :-2,
'lags' :-2,
'lapse' :-2,
'lapsing' :-2,
'laundering' :-2,
'limitations' :-2,
'lingering' :-2,
'liquidate' :-2,
'liquidated' :-2,
'liquidates' :-2,
'liquidation' :-2,
'liquidations' :-2,
'litigating' :-2,
'litigation' :-2,
'lose' :-2,
'loses' :-2,
'losing' :-2,
'loss' :-2,
'losses' :-2,
'lost' :-2,
'lying' :-2,
'malfeasance' :-2,
'malfunction' :-2,
'malfunctioning' :-2,
'malfunctions' :-2,
'malicious' :-2,
'malpractice' :-2,
'manipulate' :-2,
'manipulated' :-2,
'manipulation' :-2,
'markdowns' :-2,
'misappropriating' :-2,
'misappropriation' :-2,
'miscalculations' :-2,
'mishandling' :-2,
'misjudge' :-2,
'misjudged' :-2,
'misjudges' :-2,
'misjudgment' :-2,
'misjudgments' :-2,
'misleading' :-2,
'mismanagement' :-2,
'mismatch' :-2,
'mismatching' :-2,
'misplaced' :-2,
'mispricing' :-2,
'mispricings' :-2,
'misrepresentation' :-2,
'miss' :-2,
'missed' :-2,
'misses' :-2,
'mistaken' :-2,
'mistakes' :-2,
'misunderstood' :-2,
'misuse' :-2,
'moratoria' :-2,
'moratorium' :-2,
'moratoriums' :-2,
'negative' :-2,
'negatively' :-2,
'neglect' :-2,
'negligence' :-2,
'negligent' :-2,
'noncompliant' :-2,
'nonpayment' :-2,
'nonperformance' :-2,
'nonperforming' :-2,
'nullify' :-2,
'nullifying' :-2,
'obsolescence' :-2,
'obsolete' :-2,
'obstacles' :-2,
'omission' :-2,
'omissions' :-2,
'omit' :-2,
'omitted' :-2,
'omitting' :-2,
'onerous' :-2,
'oppose' :-2,
'opposed' :-2,
'opposition' :-2,
'outages' :-2,
'outdated' :-2,
'overbuilding' :-2,
'overburdened' :-2,
'overcapacity' :-2,
'overestimates' :-2,
'overestimation' :-2,
'overlooked' :-2,
'overproduction' :-2,
'overruns' :-2,
'overstate' :-2,
'overstated' :-2,
'oversupply' :-2,
'overturned' :-2,
'overvalued' :-2,
'panic' :-2,
'penalties' :-2,
'penalty' :-2,
'peril' :-2,
'perjury' :-2,
'persist' :-2,
'persisted' :-2,
'persistence' :-2,
'persistent' :-2,
'persistently' :-2,
'persists' :-2,
'pervasive' :-2,
'pervasiveness' :-2,
'poor' :-2,
'poorly' :-2,
'poses' :-2,
'posing' :-2,
'postpone' :-2,
'postponed' :-2,
'postponing' :-2,
'preclude' :-2,
'precludes' :-2,
'precluding' :-2,
'premature' :-2,
'preventing' :-2,
'prevention' :-2,
'prevents' :-2,
'problem' :-2,
'problematic' :-2,
'problems' :-2,
'prolongation' :-2,
'prolonged' :-2,
'prone' :-2,
'protests' :-2,
'protracted' :-2,
'punitive' :-2,
'purport' :-2,
'purports' :-2,
'question' :-2,
'questionable' :-2,
'questions' :-2,
'recall' :-2,
'recalls' :-2,
'recession' :-2,
'recessionary' :-2,
'recessions' :-2,
'redress' :-2,
'refusal' :-2,
'refuse' :-2,
'refused' :-2,
'refuses' :-2,
'reject' :-2,
'rejected' :-2,
'rejection' :-2,
'relinquishes' :-2,
'reluctance' :-2,
'reluctant' :-2,
'repossessed' :-2,
'repudiate' :-2,
'repudiation' :-2,
'resign' :-2,
'resignation' :-2,
'restate' :-2,
'restated' :-2,
'restatement' :-2,
'restatements' :-2,
'restating' :-2,
'restructure' :-2,
'restructured' :-2,
'restructures' :-2,
'restructuring' :-2,
'restructurings' :-2,
'retaliatory' :-2,
'revocation' :-2,
'revoke' :-2,
'revoked' :-2,
'revoking' :-2,
'riskier' :-2,
'riskiest' :-2,
'risky' :-2,
'sacrifice' :-2,
'sacrificed' :-2,
'scrutiny' :-2,
'seize' :-2,
'seized' :-2,
'serious' :-2,
'seriously' :-2,
'setback' :-2,
'setbacks' :-2,
'severe' :-2,
'severely' :-2,
'severity' :-2,
'sharply' :-2,
'shortage' :-2,
'shortages' :-2,
'shortfall' :-2,
'shortfalls' :-2,
'shut' :-2,
'shutdown' :-2,
'slippage' :-2,
'slow' :-2,
'slowdown' :-2,
'slowdowns' :-2,
'slowed' :-2,
'slower' :-2,
'slowing' :-2,
'slowly' :-2,
'sluggish' :-2,
'stagnation' :-2,
'stolen' :-2,
'stoppages' :-2,
'strain' :-2,
'strained' :-2,
'strains' :-2,
'stress' :-2,
'stressed' :-2,
'stresses' :-2,
'subjecting' :-2,
'sue' :-2,
'suffer' :-2,
'suffered' :-2,
'suffering' :-2,
'suffers' :-2,
'susceptibility' :-2,
'susceptible' :-2,
'suspect' :-2,
'suspend' :-2,
'suspended' :-2,
'suspending' :-2,
'suspension' :-2,
'suspensions' :-2,
'tense' :-2,
'terminate' :-2,
'terminated' :-2,
'terminates' :-2,
'terminating' :-2,
'termination' :-2,
'threat' :-2,
'threaten' :-2,
'threatened' :-2,
'threatens' :-2,
'threats' :-2,
'tightening' :-2,
'tolerate' :-2,
'trouble' :-2,
'troubled' :-2,
'troubles' :-2,
'turbulence' :-2,
'turmoil' :-2,
'unable' :-2,
'unanticipated' :-2,
'unattractive' :-2,
'unauthorized' :-2,
'unavailability' :-2,
'unavailable' :-2,
'unavoidable' :-2,
'unaware' :-2,
'uncontrollable' :-2,
'undelivered' :-2,
'undercapitalized' :-2,
'underestimate' :-2,
'underestimated' :-2,
'underfunded' :-2,
'undermine' :-2,
'underperform' :-2,
'underperformance' :-2,
'underperformed' :-2,
'underperforming' :-2,
'underperforms' :-2,
'underproduction' :-2,
'understate' :-2,
'underutilization' :-2,
'undesirable' :-2,
'undetected' :-2,
'undetermined' :-2,
'undisclosed' :-2,
'undue' :-2,
'uneconomic' :-2,
'uneconomical' :-2,
'unemployed' :-2,
'unemployment' :-2,
'unethical' :-2,
'unexpected' :-2,
'unexpectedly' :-2,
'unfair' :-2,
'unfavorable' :-2,
'unfavorably' :-2,
'unfavourable' :-2,
'unforeseeable' :-2,
'unforeseen' :-2,
'unfounded' :-2,
'unfunded' :-2,
'uninsured' :-2,
'unintended' :-2,
'unintentional' :-2,
'unintentionally' :-2,
'unjustified' :-2,
'unlawful' :-2,
'unnecessary' :-2,
'unpaid' :-2,
'unplanned' :-2,
'unpredictability' :-2,
'unpredictable' :-2,
'unpredictably' :-2,
'unprofitable' :-2,
'unreasonable' :-2,
'unreliable' :-2,
'unresolved' :-2,
'unrest' :-2,
'unsafe' :-2,
'unstable' :-2,
'unsuccessful' :-2,
'untimely' :-2,
'unusable' :-2,
'unwarranted' :-2,
'unwilling' :-2,
'unwillingness' :-2,
'upset' :-2,
'usury' :-2,
'violate' :-2,
'violated' :-2,
'violates' :-2,
'violating' :-2,
'violation' :-2,
'violations' :-2,
'violators' :-2,
'violence' :-2,
'violent' :-2,
'voided' :-2,
'volatile' :-2,
'volatility' :-2,
'vulnerabilities' :-2,
'vulnerability' :-2,
'vulnerable' :-2,
'warned' :-2,
'warning' :-2,
'warnings' :-2,
'wasting' :-2,
'weak' :-2,
'weaken' :-2,
'weakened' :-2,
'weakening' :-2,
'weakens' :-2,
'weaker' :-2,
'weakness' :-2,
'weaknesses' :-2,
'worse' :-2,
'worsen' :-2,
'worsened' :-2,
'worsening' :-2,
'worsens' :-2,
'worst' :-2,
'worthless' :-2,
'wrong' :-2,
'wrongfully' :-2,
'accomplish' :2,
'accomplishing' :2,
'accomplishment' :2,
'achieve' :2,
'achieved' :2,
'achievement' :2,
'achievements' :2,
'achieves' :2,
'achieving' :2,
'adequately' :2,
'advancement' :2,
'advancements' :2,
'advances' :2,
'advancing' :2,
'advantage' :2,
'advantaged' :2,
'advantageous' :2,
'advantageously' :2,
'advantages' :2,
'assure' :2,
'assured' :2,
'attain' :2,
'attained' :2,
'attaining' :2,
'attainment' :2,
'attractive' :2,
'attractiveness' :2,
'beneficial' :2,
'beneficially' :2,
'benefit' :2,
'benefited' :2,
'benefiting' :2,
'benefitted' :2,
'benefitting' :2,
'best' :2,
'better' :2,
'booming' :2,
'boost' :2,
'breakthroughs' :2,
'charitable' :2,
'collaborate' :2,
'collaboration' :2,
'collaborative' :2,
'compliment' :2,
'complimentary' :2,
'confident' :2,
'constructive' :2,
'constructively' :2,
'dependable' :2,
'desirable' :2,
'desired' :2,
'easier' :2,
'easily' :2,
'easy' :2,
'effective' :2,
'efficiencies' :2,
'efficiency' :2,
'efficient' :2,
'efficiently' :2,
'empower' :2,
'enable' :2,
'enabled' :2,
'enables' :2,
'enabling' :2,
'encouraged' :2,
'encourages' :2,
'encouraging' :2,
'enhance' :2,
'enhanced' :2,
'enhancement' :2,
'enhancements' :2,
'enhances' :2,
'enhancing' :2,
'enjoy' :2,
'enjoyed' :2,
'excellent' :2,
'exceptional' :2,
'exceptionally' :2,
'favorable' :2,
'favorably' :2,
'favored' :2,
'favoring' :2,
'friendly' :2,
'gain' :2,
'gained' :2,
'gaining' :2,
'gains' :2,
'good' :2,
'great' :2,
'greater' :2,
'greatest' :2,
'greatly' :2,
'highest' :2,
'honor' :2,
'honoring' :2,
'ideal' :2,
'improve' :2,
'improved' :2,
'improvement' :2,
'improvements' :2,
'improves' :2,
'improving' :2,
'influential' :2,
'innovate' :2,
'innovation' :2,
'innovations' :2,
'innovative' :2,
'innovators' :2,
'integrity' :2,
'inventing' :2,
'inventions' :2,
'leadership' :2,
'leading' :2,
'loyal' :2,
'lucrative' :2,
'mitigate' :2,
'opportunities' :2,
'opportunistic' :2,
'opportunistically' :2,
'opportunity' :2,
'optimistic' :2,
'outperform' :2,
'outperformed' :2,
'outperforming' :2,
'outperforms' :2,
'overcome' :2,
'perfect' :2,
'perfectly' :2,
'positive' :2,
'positively' :2,
'premier' :2,
'profitability' :2,
'profitable' :2,
'profitably' :2,
'progress' :2,
'prosperity' :2,
'rebound' :2,
'rebounded' :2,
'rebounding' :2,
'resolve' :2,
'reward' :2,
'rewarded' :2,
'rewards' :2,
'satisfaction' :2,
'satisfactory' :2,
'satisfied' :2,
'satisfies' :2,
'satisfy' :2,
'satisfying' :2,
'stability' :2,
'stabilization' :2,
'stabilize' :2,
'stabilizing' :2,
'stable' :2,
'strength' :2,
'strengthen' :2,
'strengthened' :2,
'strengthening' :2,
'strengthens' :2,
'strengths' :2,
'strong' :2,
'stronger' :2,
'strongest' :2,
'succeed' :2,
'succeeded' :2,
'succeeding' :2,
'succeeds' :2,
'success' :2,
'successes' :2,
'successful' :2,
'successfully' :2,
'superior' :2,
'surpassing' :2,
'transparency' :2,
'upturn' :2,
'upturns' :2,
'valuable' :2,
'win' :2,
'winner' :2,
'winning' :2,
'worthy' :2
}

In [19]:
sia.lexicon.update(new_words)

In [20]:
for name, age in sia.lexicon.items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
    if name == 'greatest':
        print(age)

2


In [21]:
compound=[]
negative=[]
positive=[]
neutral=[]
for rows in test['Sentences']:
    compound.append(sia.polarity_scores(rows).get('compound'))
    negative.append(sia.polarity_scores(rows).get('neg'))
    positive.append(sia.polarity_scores(rows).get('pos'))
    neutral.append(sia.polarity_scores(rows).get('neu'))

In [22]:
test['negative_score']=''
test['positive_score']=''
test['neutral_score']=''
test['compound_score']=''

In [23]:
test['negative_score']=negative
test['positive_score']=positive
test['neutral_score']=neutral
test['compound_score']=compound

In [24]:
test.head()

,Sentences,Sentiment,negative_score,positive_score,neutral_score,compound_score
0,some of the portfolios holdings may underperfo...,1,0.250,0.0,0.750,-0.4588
1,the portfolio may make investments that become...,1,0.150,0.0,0.850,-0.4588
2,"as a result, stock of mid- and small-capitaliz...",1,0.333,0.0,0.667,-0.7184
3,investment risk is the risk that the portfolio...,1,0.125,0.0,0.875,-0.4588
4,small cap investing,0,0.000,0.0,1.000,0.0000


In [25]:
test['Pred_Sentiment']=np.where(test['compound_score']>=0, 0,1)

In [26]:
sum(test['Pred_Sentiment']==test['Sentiment'])/len(test['Sentiment'])

0.6777843954734961

In [27]:
test.shape

(1679, 7)

#### Taking Count of Positive and Negative Words

In [28]:
Negative_words = pd.read_excel('Negative_Words.xlsx')
Negative_words = Negative_words.apply(lambda x: x.astype(str).str.lower())
list_Negative = list(Negative_words['Negative'])

In [29]:
Positive_words = pd.read_csv('Positive_Words.csv')
Positive_words = Positive_words.apply(lambda x: x.astype(str).str.lower())
list_Positive = list(Positive_words['Positive'])

In [30]:
count_pos=0
length_tokens=0
cnt_pos=[]
for row in test['Sentences']:
    tokens=nltk.word_tokenize(row)
    tokens1=[word.lower() for word in tokens]
    length_tokens=len(tokens1)
    #count_neg.append(len([i for i in tokens1 if i in list_Negative]))
    count_pos = len([i for i in tokens1 if i in list_Positive])
    cnt_pos.append(count_pos)

    
test['Pos_Words']=''
test['Pos_Words']=cnt_pos

In [31]:
count_neg=0
length_tokens=0
cnt_neg=[]
for row in test['Sentences']:
    tokens=nltk.word_tokenize(row)
    tokens1=[word.lower() for word in tokens]
    length_tokens=len(tokens1)
    #count_neg.append(len([i for i in tokens1 if i in list_Negative]))
    count_neg = len([i for i in tokens1 if i in list_Negative])
    cnt_neg.append(count_neg)
    
test['Neg_Words']=''
test['Neg_Words']=cnt_neg

In [32]:
conditions = [(test['compound_score']>=0) & (test['Pos_Words']>test['Neg_Words']),(test['compound_score']<0) & (test['Pos_Words']<test['Neg_Words'])]

In [33]:
choices=[0,1]

In [34]:
test['Pred_Sentiment2']=np.select(conditions, choices)

In [35]:
test.head()

,Sentences,Sentiment,negative_score,positive_score,neutral_score,compound_score,Pred_Sentiment,Pos_Words,Neg_Words,Pred_Sentiment2
0,some of the portfolios holdings may underperfo...,1,0.250,0.0,0.750,-0.4588,1,0,1,1
1,the portfolio may make investments that become...,1,0.150,0.0,0.850,-0.4588,1,0,1,1
2,"as a result, stock of mid- and small-capitaliz...",1,0.333,0.0,0.667,-0.7184,1,0,2,1
3,investment risk is the risk that the portfolio...,1,0.125,0.0,0.875,-0.4588,1,0,1,1
4,small cap investing,0,0.000,0.0,1.000,0.0000,0,0,0,0


In [36]:
sum(test['Pred_Sentiment2']==test['Sentiment'])/len(test['Sentiment'])

0.6861226920786182

# Random Checks

In [43]:
missclassified=test.loc[(test['Sentiment']!=test['Pred_Sentiment'])]

In [44]:
missclassified.shape

(541, 10)

In [139]:
missclassified.head()

,Sentences,Sentiment,negative,positive,neutral,compound,Pred_Sentiment,Pos_Words,Neg_Words,Pred_Sentiment2
17,currency risk is especially high in emerging m...,1,0.000,0.0,1.000,0.0000,0,0,0,0
27,foreign investmentsinvesting in foreign (non-u...,0,0.052,0.0,0.948,-0.4588,1,0,1,1
36,"the stock market tends to be cyclical, with pe...",0,0.130,0.0,0.870,-0.4588,1,0,1,1
38,"from time to time, the stock market may not fa...",1,0.000,0.0,1.000,0.0000,0,0,0,0
45,other investment companiesthe main risk of inv...,1,0.000,0.0,1.000,0.0000,0,0,0,0


In [142]:
missclassified.to_csv('missclassified_testdata.csv', index=False)

In [171]:
correct_classified=test.loc[(test['Sentiment']==test['Pred_Sentiment2'])]

In [172]:
correct_classified.shape

(1092, 10)

In [173]:
test.shape

(1679, 10)

In [174]:
correct_classified.drop(columns=['negative','positive','neutral','compound','Pred_Sentiment','Pos_Words','Neg_Words','Pred_Sentiment2'], inplace=True)

In [175]:
correct_classified.head()

,Sentences,Sentiment
3,some of the portfolios holdings may underperfo...,1
33,the portfolio may make investments that become...,1
44,"as a result, stock of mid- and small-capitaliz...",1
48,investment risk is the risk that the portfolio...,1
58,small cap investing,0


In [176]:
relabelled=pd.read_csv('Relabelled_Misclassified_Test.csv')

In [177]:
relabelled.shape

(587, 2)

In [179]:
relabelled.head()

,Sentences,Sentiment
0,currency risk is especially high in emerging m...,1
1,foreign investmentsinvesting in foreign (non-u...,0
2,"the stock market tends to be cyclical, with pe...",0
3,"from time to time, the stock market may not fa...",0
4,other investment companiesthe main risk of inv...,0


In [180]:
final=correct_classified.append(relabelled)

In [181]:
final.shape

(1679, 2)

In [182]:
final.head()

,Sentences,Sentiment
3,some of the portfolios holdings may underperfo...,1
33,the portfolio may make investments that become...,1
44,"as a result, stock of mid- and small-capitaliz...",1
48,investment risk is the risk that the portfolio...,1
58,small cap investing,0


In [183]:
final.to_csv('Final_Relabelled_8thApril.csv', index=False)